In [1]:
%matplotlib inline
import torch
import torch.nn.functional as F
import os, sys
from pathlib import Path
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython import display
import ipywidgets as widgets
from PIL import Image
import io
import cv2
import time

In [2]:
model_path = Path('./TCN/mnist_pixel/models')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

print(model_path / (model_name+'.pt'))

_, test_loader = data_generator(data_path, batch_size)
model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')))
model.eval()

print(model.receptive_field)

if torch.cuda.is_available():
    model.cuda()
    
model.fast_inference(batch_size)

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=40, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1)
TCN/mnist_pixel/models/k7l6.pt
757


In [3]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

curr_im = np.ones((28,28), dtype=np.uint8)*255
_, encoded_image = cv2.imencode('.png', curr_im)
im_bytes = encoded_image.tobytes()
im_disp = widgets.Image(value=im_bytes, width=200, height=200)

true_val = widgets.Label(value=f'True Label: N/A')
pred_val = widgets.Label(value=f'Predicted Label: N/A')
label_disp = widgets.VBox((true_val, pred_val))

display.display(im_disp)
display.display(label_disp)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = (im/im.max() * 255).astype('uint8')
    curr_im = np.ones(im.shape, dtype=np.uint8)*255
    
    true_val.value = f'True Label: {target.item()}'

    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        curr_im[curr_row,curr_col] = im[curr_row,curr_col]
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        
    
        output = model.single_forward(data[:,:,i].view(data.size()[0], data.size()[1], 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
    
    time.sleep(2)
    
    curr_im = np.zeros(im.shape, dtype=np.uint8)
    _, encoded_image = cv2.imencode('.png', curr_im)
    im_bytes = encoded_image.tobytes()
    im_disp.value = im_bytes
    true_val.value = f'True Label: N/A'
    for i in np.zeros(np.random.randint(50, 200)):
        num_samples += 1
        output = model.single_forward(torch.tensor([i], dtype=torch.float).cuda().view(1, 1, 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
        


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

KeyboardInterrupt: 

In [169]:
test = Image.fromarray((im*255).astype('uint8'),'L')


In [170]:
display.display(test)

In [180]:
_, encoded_image = cv2.imencode('.png', (im*255).astype('uint8'))
im_bytes = encoded_image.tobytes()


widgets.Image(value=im_bytes, width=200, height=200)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

In [ ]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = im/im.max()
    curr_im = np.ones(im.shape)
    
#     plt.imshow(curr_im, cmap='gray', vmin=0, vmax=1)
#     plt.axis('off')


    true_val = widgets.Label(value=f'True Label: {target.item()}')
    pred_val = widgets.Label(value=f'Predicted Label: N/A')
    label_disp = widgets.VBox((true_val, pred_val))
    display.display(label_disp)
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        curr_im[curr_row,curr_col] = im[curr_row,curr_col]
        
    
#         if i % 20 == 0:
#             display.display(plt.gcf())
#             display.clear_output(wait=True)
#             plt.imshow(curr_im, cmap='gray', vmin=0, vmax=1)
#             plt.axis('off')
#             plt.pause(0.001)
        
        output = model.single_forward(data[:,:,i].view(data.size()[0], data.size()[1], 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        pred_val.value = f'Predicted Label: {pred.item()}'
    
#     display.display(plt.gcf())
#     display.clear_output(wait=True)
#     plt.imshow(curr_im, cmap='gray', vmin=0, vmax=1)
#     plt.axis('off')
#     plt.pause(0.001)

    

    break